<a href="https://colab.research.google.com/github/Taedriel/ZSL-v2/blob/wordEmbedding/WordsEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


In [11]:
!pip install transformers wget 
!pwd

/content


In [2]:
import wget, os

import torch
from transformers import BertTokenizer, BertModel

import logging
logging.basicConfig(level = logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
logging.info("Downloading dataset...")

url = ""

INFO:root:Downloading dataset...


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', padding=True, truncation=True,)

model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states = True, )

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
tags = ["dog", "cat", "mouse", "frog", "horse", "cow"]

vectors = []

with open("vectors.csv", "w") as f:
    for tag in tags:

        inputs = tokenizer(tag, return_tensors = "pt")

        with torch.no_grad():
            outputs = model(**inputs)

        hidden_states = outputs[2]

# print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
# layer_i = 0

# print ("Number of batches:", len(hidden_states[layer_i]))
# batch_i = 0

# print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
# token_i = 0

# print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

# We got:
# [# layers, # batches, # tokens, # features]
# Desired dimensions:
# [# tokens, # layers, # features]

# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
        token_embeddings = torch.stack(hidden_states, dim=0)
# Remove dimension 1, the "batches".
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
# Swap dimensions 0 and 1.
        token_embeddings = token_embeddings.permute(1,0,2)

# apply different strategy to summarize word embeddings
        token_sum_4 = []
        for token in token_embeddings:

            sum_vec = torch.sum(token[-4:], dim = 0)
            token_sum_4.append(sum_vec)


        line = ",".join(list(map(str, map(float, token_sum_4[1][:]))))
        vectors.append((tag, token_sum_4[1][:]))
        print(tag, ",", line, sep="", file=f)

for vector in vectors:

    for otherVector in vectors:

        if vector[0] == otherVector[0]:
            continue
        
        print(vector[1])
        print(otherVector[1])
        similarity = torch.cosine_similarity(vector[1], otherVector[1])

        print(vector[0], "^", otherVector[0], "=", similarity)


tensor([ 3.5881e-01,  1.7862e+00, -3.1899e-01, -1.9114e+00,  7.8711e-01,
         2.4177e+00,  1.4130e+00, -7.4377e-01, -4.8974e-01, -1.6070e+00,
         1.6664e-02,  1.3953e+00,  1.2188e+00, -1.5406e+00, -3.0088e+00,
         6.1588e-01, -3.5549e-01,  1.4884e+00,  9.3974e-01,  3.7884e+00,
        -1.9804e-01, -1.9601e+00, -3.4868e+00, -4.4936e+00,  1.3606e+00,
         2.9788e+00,  2.4246e+00,  3.4250e+00, -1.4945e+00,  1.5956e+00,
         2.4130e+00, -1.1299e+00,  2.6815e+00,  9.9327e-01, -6.7634e-01,
        -1.5427e-01, -1.3657e+00, -1.0259e+00, -3.5282e+00,  1.4723e+00,
         4.0991e+00,  2.2901e-02,  2.0842e+00, -1.7460e+00,  3.4732e+00,
         1.7389e+00,  1.2036e+00, -7.3772e-01,  1.9450e+00, -3.1480e+00,
        -2.3191e+00,  9.4697e-01, -1.0936e+00, -4.4956e-02,  6.7191e-02,
        -2.0033e+00,  3.8751e+00, -2.2753e-01, -1.6732e+00, -1.8374e+00,
        -2.8086e+00, -1.8833e+00,  1.0801e+00, -1.0530e+00, -9.7709e-01,
         2.1032e+00,  3.0946e+00,  5.5217e-01, -4.3

IndexError: ignored